## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant

___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

In [1]:
## Pardon the messy codes and comments due to iterations of trial-and-error and to remind myself what's going on.

import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score, recall_score, precision_score
from sklearn.preprocessing import LabelEncoder

def blight_model():
    
    df1 = pd.read_csv("train.csv", encoding='latin1')
    
    df_address = pd.read_csv("addresses.csv", encoding='latin1', index_col='address')
    df_latlon = pd.read_csv("latlons.csv", index_col='address')
    df4 = df_address.join(df_latlon)
    df_main = df4.merge(df1, left_on='ticket_id', right_on='ticket_id').set_index('ticket_id')
    
    # Adding days between ticket issued and hearing dates as a parameter.
    def gapdays(hearing_dt, ticket_dt):
        try:
            hearing_datetime = datetime.strptime(hearing_dt, '%Y-%m-%d %H:%M:%S')
            ticket_datetime = datetime.strptime(ticket_dt, '%Y-%m-%d %H:%M:%S')
            diff = (hearing_datetime - ticket_datetime).days
        except:
            diff = 0
        return diff
    
    df_main['gapdays'] = df_main.apply(lambda row: gapdays(row['hearing_date'], row['ticket_issued_date']), axis=1)

    # Dropped from feature selection.
    # df_main['violation_class'] = LabelEncoder().fit_transform(df_main['violation_code'])
    # Replace 'disposition' with 3 binary features: 'not_responsible', 'pending' and 'responsible'.
    # df_main['not_responsible'] = 0
    # df_main['not_responsible'][df_main['disposition']
    #    .isin(['Not responsible by Dismissal', 
    #           'Not responsible by City Dismissal',
    #           'Not responsible by Determination'])] = 1
    # df_main['pending'] = 0
    # df_main['pending'][df_main['disposition']
    #    .isin(['PENDING JUDGMENT',
    #           'SET-ASIDE (PENDING JUDGMENT)'])] = 1
    # df_main['responsible'] = 0
    # df_main['responsible'][df_main['disposition']
    #    .isin(['Responsible by Default',
    #           'Responsible by Determination',
    #           'Responsible by Admission', 
    #           'Responsible (Fine Waived) by Deter'])] = 1
    
#      All Features:
#      ['lat', 'lon', 'agency_name', 'inspector_name', 'violator_name','violation_street_number', 'violation_street_name',
#       'violation_zip_code', 'mailing_address_str_number', 'mailing_address_str_name', 'city', 'state', 'zip_code',
#       'non_us_str_code', 'country', 'ticket_issued_date', 'hearing_date', 'violation_code', 'violation_description', 'disposition', 'fine_amount',
#       'admin_fee', 'state_fee', 'late_fee', 'discount_amount', 'clean_up_cost', 'judgment_amount', 'payment_amount', 'balance_due',
#       'payment_date', 'payment_status', 'collection_status', 'grafitti_status', 'compliance_detail']
    
    # Feature selection  
    selected_features = ['lat','lon', 'judgment_amount', 'discount_amount', 'gapdays', 'compliance']
    df_main = df_main[selected_features]
    
    # Removing those NaN, including those who are found not responsible.
    df = df_main.dropna()
    
    X = df.iloc[:,:(df.shape[1]-1)]
    y = df.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    clf = RandomForestRegressor(n_estimators=100, n_jobs=-1).fit(X_train, y_train)
    
    ## Feature Selection
    # clf.feature_importances_
    # ['lat','lon', 'violation_class', 'judgment_amount', 'discount_amount', 'gapdays', 'compliance']
    # array([ 0.30768009,  0.30742091,  0.02660175,  0.10365726,  0.08440342,  0.17023657])
    
    ## Parameters Tuning
    # gridvals = {'n_estimators':[40,60,80,100]}
    # gridsearch_rec = GridSearchCV(clf, param_grid=gridvals, cv=3, scoring='roc_auc')
    # gridsearch_rec.fit(X_train, y_train)
    # return gridsearch_rec.cv_results_['mean_test_score']
    # >>array([ 0.77262559,  0.77424137,  0.7775128 ,  0.778432  ])
       
    # y_pred_score = clf.predict(X_test)
    # fpr, tpr, _ = roc_curve(y_test, y_pred_score)
    # roc_auc = auc(fpr, tpr)
    # roc_auc, accuracy_score(y_test, y_pred_score), precision_score(y_test, y_pred_score), recall_score(y_test, y_pred_score)
    # >> Returns an auc score of more than 0.75
    
    df_testcsv = pd.read_csv('test.csv', encoding='latin1')
    df_testcsv = df4.merge(df_testcsv, left_on='ticket_id', right_on='ticket_id').set_index('ticket_id')
    df_testcsv['gapdays'] = df_testcsv.apply(lambda row: gapdays(row['hearing_date'], row['ticket_issued_date']), axis=1)
    df_test = df_testcsv[['lat','lon', 'judgment_amount', 'discount_amount', 'gapdays']]
    df_test = df_test.fillna(df_test.mean())
    
    df_test['compliance_prob'] = clf.predict(df_test)
    
    return df_test['compliance_prob']

In [13]:
blight_model()

In [2]:
#Investigate discount_amount here. 
import pandas as pd
df1 = pd.read_csv("train.csv", encoding='latin1')
    
df_address = pd.read_csv("addresses.csv", encoding='latin1', index_col='address')
df_latlon = pd.read_csv("latlons.csv", index_col='address')
df4 = df_address.join(df_latlon)
df_main = df4.merge(df1, left_on='ticket_id', right_on='ticket_id').set_index('ticket_id')

df_main.groupby(['discount_amount','compliance']).size()
# Conclusion: discount_amount incentivise people to improve compliance.

discount_amount  compliance
0.0              0.0           148226
                 1.0            10474
3.0              1.0               19
5.0              0.0                8
                 1.0              159
10.0             0.0               14
                 1.0              141
13.0             1.0                1
20.0             0.0               13
                 1.0              122
25.0             0.0               18
                 1.0              587
30.0             1.0               17
40.0             1.0                1
50.0             0.0                3
                 1.0               40
100.0            1.0               16
250.0            0.0                1
                 1.0                5
350.0            1.0               15
dtype: int64

In [26]:
#Investigating 'violation_code' here
df_main['violation_code'].nunique()
df_main[['violation_code', 'compliance']].groupby(['violation_code', 'compliance']).size()

# Conclusion: not a lot of difference in compliance across various violation_codes

violation_code        compliance
19420901              0.0              12
                      1.0               4
19450901              0.0            1575
                      1.0              45
19830901              0.0              10
19840901              0.0               1
                      1.0               1
19850901              0.0               1
20130901              0.0              75
                      1.0               3
20160901              0.0               1
20180901              0.0              13
                      1.0               2
22-2-16               0.0              17
22-2-17               0.0            1314
                      1.0             100
22-2-17(a)            0.0              63
                      1.0               8
22-2-18               0.0               2
22-2-20               0.0               2
                      1.0               1
22-2-21(a)            0.0               2
22-2-21(b)            0.0             162
 

In [92]:
#Investigating 'disposition' as a feature.
df_main['disposition'].unique().tolist()
# >> 'Not responsible by Dismissal',
# >> 'Not responsible by City Dismissal',
# >> 'PENDING JUDGMENT',
# >> 'Responsible by Default',
# >> 'Responsible by Admission',
# >> 'Not responsible by Determination',
# >> 'Responsible by Determination',
# >> 'SET-ASIDE (PENDING JUDGMENT)',
# >> 'Responsible (Fine Waived) by Deter'
df_main['not_responsible'] = 0
df_main['not_responsible'][df_main['disposition']
    .isin(['Not responsible by Dismissal', 
           'Not responsible by City Dismissal',
           'Not responsible by Determination'])] = 1
df_main['pending'] = 0
df_main['pending'][df_main['disposition']
    .isin(['PENDING JUDGMENT',
           'SET-ASIDE (PENDING JUDGMENT)'])] = 1
df_main['responsible'] = 0
df_main['responsible'][df_main['disposition']
    .isin(['Responsible by Default',
           'Responsible by Determination',
           'Responsible by Admission', 
           'Responsible (Fine Waived) by Deter'])] = 1

df_main.groupby('not_responsible').size()

not_responsible
0    160571
1     89735
dtype: int64